In [ ]:
import pandas as pd
import numpy as np

In [ ]:
filename = 'ANONYMOUS_DATA_202210_60002'
df = pd.read_csv(f'../data/raw_data_unzipped/{filename}.csv',)

In [ ]:
# remove rows with missing values
hourly_cols = df.iloc[:, 7:55]
df = df[~hourly_cols.isna().any(axis=1)]

In [ ]:
df['INTERVAL_READING_DATE'] = pd.to_datetime(df['INTERVAL_READING_DATE'])
df['DAY'] = df['INTERVAL_READING_DATE'].dt.day

In [ ]:
# keep only full months users
user_date_counts = (
    df.groupby('ACCOUNT_IDENTIFIER')['INTERVAL_READING_DATE']
    .count()
    .reset_index(name='DATE_COUNT')
)

full_month_users = (
    user_date_counts
    .query('DATE_COUNT == 31')
    ['ACCOUNT_IDENTIFIER']
    .tolist()

)

df = (
    df[df.ACCOUNT_IDENTIFIER.isin(full_month_users)]
)


In [ ]:
reshaped_user_data = []
time_interval_columns = [
    f"INTERVAL_HR{(30 * i) // 60:02d}{(30 * i) % 60:02d}_ENERGY_QTY"
    for i in range(1, 49)
]

for user in full_month_users:
    user_df = (
        df.query('ACCOUNT_IDENTIFIER == @user')
        .sort_values('DAY')
    )
    reshaped_user_data.append(
        user_df[['DAY']+time_interval_columns].values.reshape(-1, 1)
    )
reshaped_user_data = np.hstack(reshaped_user_data)

np.save(f'{filename}.npy', reshaped_user_data)


In [ ]:
reshaped_user_data

In [ ]:
time_interval_columns = [
    f"INTERVAL_HR{(30 * i) // 60:02d}{(30 * i) % 60:02d}_ENERGY_QTY"
    for i in range(1, 49)
]

In [ ]:
interval_columns

In [ ]:
A = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16],
    [17, 18, 19, 20],
    [21, 22, 23, 24],
    [25, 26, 27, 28],
    [29, 30, 31, 32]
])

In [ ]:
A[::2, :]

In [ ]:

hour=2
step = int(2*hour)
row_size, col_size = int(A.shape[0] // step), int(A.shape[1])
A_hour=np.zeros((row_size, col_size))

for i in range(step):
    A_hour += A[i::step, :]


In [ ]:
A.shape[1] // hour

In [ ]:
int(A.shape[0] // hour)

In [ ]:
A=A[:,0].reshape(-1, 1)


In [ ]:
r=3
data_with_previous_r_intervals = []
for i in range(A.shape[1]):
    col_with_r_intervals = []
    for j in range(A.shape[0]-r):
        col_with_r_intervals.append(A[j:j+r, i].reshape(1, -1))
    col_with_r_intervals = np.vstack(col_with_r_intervals)
    data_with_previous_r_intervals.append(col_with_r_intervals)
data_with_previous_r_intervals = np.hstack(data_with_previous_r_intervals)
data_with_previous_r_intervals

In [ ]:
np.concat([data_with_previous_r_intervals, A[r:,:]], axis=1)

In [ ]:
A[r,:]

In [ ]:
np.arange(20)

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from src.exception import CustomException
from src.logger import logging
from dataclasses import dataclass
from catboost import CatBoostRegressor
from sklearn.ensemble import (
    AdaBoostRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from src.utils import save_object, evaluate_models


models = {
    "Random Forest": RandomForestRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Linear Regression": LinearRegression(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor(),
}

In [ ]:
test = ["best"]
test["best"]

In [ ]:
models["Random Forest"]

In [ ]:
strategy_name={"rma_pca", "rma_rbd"}

for strategy in strategy_name:
    print(strategy)